In [ ]:
import torch
import torch.nn as nn

class CNN_LSTM_Model(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, cnn_out_channels, kernel_size, output_size):
        super(CNN_LSTM_Model, self).__init__()
        
        # CNN layers
        self.cnn = nn.Conv1d(in_channels=input_size, 
                             out_channels=cnn_out_channels, 
                             kernel_size=kernel_size)
        
        # LSTM layers
        self.lstm = nn.LSTM(input_size=cnn_out_channels, 
                            hidden_size=hidden_size, 
                            num_layers=num_layers, 
                            batch_first=True)
        
        # Fully connected layer for output
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = x.permute(0, 2, 1)
        x = torch.relu(self.cnn(x))
        x = x.permute(0, 2, 1)
        x, _ = self.lstm(x)
        x = x[:, -1, :]
        x = self.fc(x)
        
        return x

# Parameters
input_size = 10  
hidden_size = 64  
num_layers = 2    
cnn_out_channels = 32  
kernel_size = 3   
output_size = 1   # Predicting the spread

# Model instantiation
model = CNN_LSTM_Model(input_size, hidden_size, num_layers, cnn_out_channels, kernel_size, output_size)

In [ ]:
class TransformerTimeSeriesModel(nn.Module):
    def __init__(self, input_size, d_model, nhead, num_encoder_layers, dim_feedforward, output_size):
        super(TransformerTimeSeriesModel, self).__init__()
        
        # Linear layer to project inputs to d_model dimensions for the Transformer
        self.input_fc = nn.Linear(input_size, d_model)
        
        # Transformer Encoder layer
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward),
            num_layers=num_encoder_layers
        )
        
        # Fully connected output layer
        self.output_fc = nn.Linear(d_model, output_size)

    def forward(self, x):
        x = self.input_fc(x)
        x = x.permute(1, 0, 2)
        x = self.transformer_encoder(x)
        x = x[-1, :, :]
        x = self.output_fc(x)
        
        return x

# Parameters
input_size = 10  
d_model = 64    
nhead = 4     
num_encoder_layers = 2 
dim_feedforward = 128  
output_size = 1        

# Model instantiation
transformer_model = TransformerTimeSeriesModel(input_size, d_model, nhead, num_encoder_layers, dim_feedforward, output_size)